In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')
import vectorbtpro as vbt
import os, sys, io, time, math, pickle, copy, json, asyncio, random, zipfile, functools, itertools
import datetime as dt
import talib.abstract as ta
import pandas as pd
import numpy as np

In [3]:
from dataloader import Dataloader
from backtest import Backtest, CommonParams, Direction

dataloader = Dataloader()
backtest = Backtest()

### Указываем пару и период с которыми будем работать

In [4]:
symbol = 'BTCUSDT'
start_date = '01.2023' # строго в формате MM.YYYY
end_date = '03.2023' # строго в формате MM.YYYY

### Скачивание исторических данных binance.futures
сохраняет csv файлы в klines/{symbol}. 
⚠️Пропустить если интересуемый период загружен

In [5]:
# dataloader.download(symbol, start_date, end_date)

### Чтение данных
Это минутные свечи в дальнейшем будут использоваться:
1. Расчет точек входа на основе индикаторов
2. Имитация торгов на исторических данных

In [6]:
df = dataloader.read(symbol, start_date, end_date)

### Сигнал

In [7]:
# Подготовка данных для получения точек входа на основе сигналов
timeframe = '1h'

signal_df = dataloader.convert_tf(df, timeframe)
ohlcv = {c: signal_df[c].values for c in signal_df.columns}

In [8]:
# Сигнал
# для примера определим точки дивергенции
rsi = ta.RSI(ohlcv, timeperiod=20)
prev_rsi = np.append(np.nan, rsi[:-1])
# close = ohlcv.get('close')
# prev_close = np.append(np.nan, close[:-1])
high = ohlcv.get('high')
prev_high = np.append(np.nan, high[:-1])
# ema = ta.EMA(ohlcv, timeperiod=20)
# prev_ema = np.append(np.nan, ema[:-1])

signal_df['signal'] = (rsi < prev_rsi) & (high > prev_high)
# быстрая проверка на наличие сигналов
print('signal count: ', signal_df['signal'].sum())

signal count:  296


### Подготовка данных для бектеста

In [9]:
bdf = df.merge(signal_df[['datetime', 'signal']], on='datetime', how='left') # signal mapping with main df

In [14]:
step = .01 # шаг перебора параметров выхода
# формируем параметры
common_params = CommonParams(
    open=bdf['open'].values,
    high=bdf['high'].values,
    low=bdf['low'].values,
    close=bdf['close'].values,
    entries=bdf['signal'].values,                                             # точки входа
    take_profits=np.arange(0.01, .10, step),                                  # take profit %
    stop_losses=np.arange(0.01, .05, step),                                   # stop loss %
    trailing_stop_activation=np.append(np.nan, np.arange(0.01, .05, step)),   # trailing stop activation % - на каком процентном росте активируется трейлинг стоп
    trailing_stops=np.append(np.nan, np.arange(0.01, .02, step/2)),           # traling stop % - если np.nan то не используется
    init_cach=10000,                                                          # начальный баланс $
    size=100,                                                                 # сумма входа в сделку $
    fees=.0004,                                                               # комиссия $
    slippage=.0001,                                                           # проскальзывание %
    direction=Direction.LONG                                                  # направление
)

In [ ]:
# запуск бектеста
cpu_usage_weight = .08 # какой процент процессоров от общего количества использовать (для ускорения)
group_size = 20 # если ОЗУ позволяет на скачках то можно увеличить (для ускорения)
result = backtest.run(common_params, cpu_usage_weight, group_size)

  0%|          | 0/27 [00:00<?, ?it/s]

### Save backtest results

In [12]:
filename = f'{symbol} {timeframe} {start_date}-{end_date}.xlsx'
result.to_excel(f'backtest_results/{filename}', index=False)

In [11]:
length = 100

# Генерация данных для цен актива
np.random.seed(0)  # Для воспроизводимости
open_prices = np.random.rand(length) * 100
close_prices = open_prices + np.random.randn(length)
low_prices = np.minimum(open_prices, close_prices) - np.random.rand(length) * 5
high_prices = np.maximum(open_prices, close_prices) + np.random.rand(length) * 5

# Создание DataFrame
df = pd.DataFrame({
    'open': open_prices,
    'high': high_prices,
    'low': low_prices,
    'close': close_prices
})

# Добавление колонки signal
signal = np.full(length, False)
signal_indices = np.random.choice(length, 5, replace=False)
signal[signal_indices] = True
df['signal'] = signal

In [11]:
pf = vbt.Portfolio.from_signals(
    close=bdf['close'], 
    open=bdf['open'], 
    high=bdf['high'], 
    low=bdf['low'], 
    entries=bdf['signal'].astype(bool),
    tp_stop=.03,
    sl_stop=.02,
    tsl_stop=.01,
    tsl_th=.01,
    init_cash=100, 
    fees=.0005, 
    slippage=.0005, 
    size=10,
    freq='1min'
)
trades = pf.get_trades().records
print('trades:', len(trades))

trades: 265
